# ASI Camera Driver Tests

This notebook tests the **ASI camera driver system** with 3 sections:

| Section | Purpose |
|---------|---------|
| **Digital Twin Test** | Tests camera workflow without hardware using `DigitalTwinCameraDriver` - discovers, connects, captures |
| **Real Hardware Test** | Same workflow but with `ASICameraDriver` for actual ZWO ASI cameras, saves capture to `/tmp/asi_test.jpg` |
| **Factory Config Test** | Tests `DriverFactory` to verify it creates correct driver types based on `DriverMode.HARDWARE` vs `DIGITAL_TWIN` |

**Key pattern:** Both drivers share identical API via the CameraRegistry abstraction - discover → get → connect → capture.

In [5]:
# Add src to path
import sys
from pathlib import Path

src_path = Path.cwd().parent / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print(f"Added {src_path} to sys.path")

Added /home/mark/src/telescope-mcp/src to sys.path


## Test with Digital Twin (No Hardware Required)

In [6]:
from telescope_mcp.drivers.cameras import DigitalTwinCameraDriver
from telescope_mcp.devices import CameraRegistry

# Use digital twin
driver = DigitalTwinCameraDriver()

with CameraRegistry(driver) as registry:
    # Discover cameras
    cameras = registry.discover()
    print(f"Found {len(cameras)} simulated camera(s)")
    
    for cam_id, info in cameras.items():
        print(f"  Camera {cam_id}: {info.name}")
    
    # Get and connect to a camera
    if cameras:
        camera = registry.get(0)
        camera.connect()
        print(f"\nConnected: {camera.is_connected}")
        print(f"Info: {camera.info}")
        
        # Capture a frame
        from telescope_mcp.devices.camera import CaptureOptions
        result = camera.capture(CaptureOptions(exposure_us=100_000))
        print(f"\nCaptured {len(result.image_data)} bytes")
        print(f"Timestamp: {result.timestamp}")
        print(f"Settings: exposure={result.exposure_us}µs, gain={result.gain}")

2025-12-24 06:02:07,427 - telescope_mcp.drivers.cameras.twin - INFO - Digital twin camera driver initialized | image_source=synthetic num_cameras=2
2025-12-24 06:02:07,430 - telescope_mcp.devices.camera - INFO - Connecting to camera | camera_id=0
2025-12-24 06:02:07,432 - telescope_mcp.drivers.cameras.twin - INFO - Opening simulated camera | camera_id=0
2025-12-24 06:02:07,434 - telescope_mcp.devices.camera - INFO - Camera connected | camera_id=0 name=Unknown resolution=0x0
2025-12-24 06:02:07,473 - telescope_mcp.devices.camera - INFO - Disconnecting camera | camera_id=0


Found 2 simulated camera(s)
  Camera 0: Camera 0
  Camera 1: Camera 1

Connected: True
Info: CameraInfo(camera_id=0, name='Unknown', max_width=0, max_height=0, is_color=False, bayer_pattern=None, supported_bins=[1], controls={'ASI_GAIN': {'Name': 'ASI_GAIN', 'MinValue': 0, 'MaxValue': 600, 'DefaultValue': 50, 'IsAutoSupported': True, 'IsWritable': True}, 'ASI_EXPOSURE': {'Name': 'ASI_EXPOSURE', 'MinValue': 0, 'MaxValue': 60000000, 'DefaultValue': 50, 'IsAutoSupported': True, 'IsWritable': True}, 'ASI_WB_R': {'Name': 'ASI_WB_R', 'MinValue': 0, 'MaxValue': 99, 'DefaultValue': 50, 'IsAutoSupported': True, 'IsWritable': True}, 'ASI_WB_B': {'Name': 'ASI_WB_B', 'MinValue': 0, 'MaxValue': 99, 'DefaultValue': 50, 'IsAutoSupported': True, 'IsWritable': True}, 'ASI_GAMMA': {'Name': 'ASI_GAMMA', 'MinValue': 0, 'MaxValue': 100, 'DefaultValue': 50, 'IsAutoSupported': True, 'IsWritable': True}, 'ASI_BRIGHTNESS': {'Name': 'ASI_BRIGHTNESS', 'MinValue': 0, 'MaxValue': 100, 'DefaultValue': 50, 'IsAutoSu

## Test with Real Hardware (Requires ASI Camera)

**Note:** Only run this if you have an ASI camera connected!

In [7]:
# Uncomment to test with real hardware

from telescope_mcp.drivers.cameras import ASICameraDriver
from telescope_mcp.devices import CameraRegistry

driver = ASICameraDriver()

with CameraRegistry(driver) as registry:
    # Discover cameras
    cameras = registry.discover()
    print(f"Found {len(cameras)} real camera(s)")
    
    for cam_id, info in cameras.items():
        print(f"  Camera {cam_id}: {info.name}")
    
    # Get and connect to a camera
    if cameras:
        camera = registry.get(0)
        camera.connect()
        print(f"\nConnected: {camera.is_connected}")
        print(f"Info: {camera.info}")
        
        # Capture a frame
        from telescope_mcp.devices.camera import CaptureOptions
        result = camera.capture(CaptureOptions(exposure_us=100_000, gain=50))
        print(f"\nCaptured {len(result.image_data)} bytes")
        
        # Save to file
        with open('/tmp/asi_test.jpg', 'wb') as f:
            f.write(result.image_data)
        print("Saved to /tmp/asi_test.jpg")

2025-12-24 06:02:07,552 - telescope_mcp.drivers.cameras.asi - INFO - ASI SDK initialized from /home/mark/src/telescope-mcp/src/telescope_mcp/drivers/asi_sdk/x64/libASICamera2.so.1.40
2025-12-24 06:02:08,894 - telescope_mcp.drivers.cameras.asi - INFO - Discovered 2 ASI camera(s)
2025-12-24 06:02:08,896 - telescope_mcp.devices.camera - INFO - Connecting to camera | camera_id=0


Found 2 real camera(s)
  Camera 0: ZWO ASI120MC-S(Camera)
  Camera 1: ZWO ASI482MC


2025-12-24 06:02:09,526 - telescope_mcp.drivers.cameras.asi - INFO - Opened ASI camera 0
2025-12-24 06:02:09,534 - telescope_mcp.devices.camera - INFO - Camera connected | camera_id=0 name="ZWO ASI120MC-S(Camera)" resolution=1280x960



Connected: True
Info: CameraInfo(camera_id=0, name='ZWO ASI120MC-S(Camera)', max_width=1280, max_height=960, is_color=True, bayer_pattern=2, supported_bins=[1, 2], controls={'Gain': {'min_value': 0, 'max_value': 100, 'default_value': 50, 'is_auto_supported': True, 'is_writable': True, 'description': 'Gain'}, 'Exposure': {'min_value': 64, 'max_value': 2000000000, 'default_value': 14000, 'is_auto_supported': True, 'is_writable': True, 'description': 'Exposure Time(us)'}, 'WB_R': {'min_value': 1, 'max_value': 100, 'default_value': 52, 'is_auto_supported': True, 'is_writable': True, 'description': 'White balance: Red component'}, 'WB_B': {'min_value': 1, 'max_value': 100, 'default_value': 95, 'is_auto_supported': True, 'is_writable': True, 'description': 'White balance: Blue component'}, 'Offset': {'min_value': 0, 'max_value': 20, 'default_value': 0, 'is_auto_supported': False, 'is_writable': True, 'description': 'offset'}, 'BandWidth': {'min_value': 40, 'max_value': 100, 'default_value':

2025-12-24 06:02:09,978 - telescope_mcp.devices.camera - INFO - Disconnecting camera | camera_id=0
2025-12-24 06:02:10,035 - telescope_mcp.drivers.cameras.asi - INFO - Closed ASI camera 0



Captured 42490 bytes
Saved to /tmp/asi_test.jpg


## Test Driver Factory Configuration

In [8]:
from telescope_mcp.drivers.config import DriverConfig, DriverMode, DriverFactory

# Test hardware mode configuration
config = DriverConfig(mode=DriverMode.HARDWARE)
factory = DriverFactory(config)

driver = factory.create_camera_driver()
print(f"Hardware mode driver: {type(driver).__name__}")

# Test digital twin mode
config = DriverConfig(mode=DriverMode.DIGITAL_TWIN)
factory = DriverFactory(config)

driver = factory.create_camera_driver()
print(f"Digital twin mode driver: {type(driver).__name__}")

2025-12-24 06:02:10,059 - telescope_mcp.drivers.cameras.twin - INFO - Digital twin camera driver initialized | image_source=synthetic num_cameras=2


Hardware mode driver: ASICameraDriver
Digital twin mode driver: DigitalTwinCameraDriver
